<a href="https://colab.research.google.com/github/junwei2110/FreeCodeCamp_MachineLearning/blob/Book-Recommendation-KNN/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-06-14 14:27:42--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M   135MB/s    in 0.2s    

2021-06-14 14:27:42 (135 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
df_books.info()
print(df_books.isna().sum())
# There is one null value for the author. No big issues as author is not one of the major variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271378 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB
isbn      0
title     0
author    1
dtype: int64


In [ ]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
df_ratings.info()
print(df_ratings.isna().sum())
# No null values. Also isbn d-type is the same as in the df_books dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB
user      0
isbn      0
rating    0
dtype: int64


In [ ]:
df_total = df_books.merge(df_ratings, left_on='isbn', right_on='isbn')
df_total.head()
# Merger completed

,isbn,title,author,user,rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,8,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,11400,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,11676,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,41385,0.0


In [ ]:
df_total.info()
# We see that there total rows is now 1,031,175 vs the total rows in df_ratings of 1,149,780. This is likely because there were some isbn that were not present in both dataframes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031175 entries, 0 to 1031174
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   isbn    1031175 non-null  object 
 1   title   1031175 non-null  object 
 2   author  1031174 non-null  object 
 3   user    1031175 non-null  int32  
 4   rating  1031175 non-null  float32
dtypes: float32(1), int32(1), object(3)
memory usage: 39.3+ MB


In [ ]:
# Calculate user and book rating counts (To filter one by one)

# Use apply, applymap or map. All 3 methods work
# df1 = df[df['A'].map(df['A'].value_counts()) > 1]
# df1 = df[df.groupby("A")['A'].transform('size') > 1]

In [ ]:
# Calculate user and book rating counts (And to filter both concurrently)

# When you only use groupby(["X1", "X2"]), nothing happens to the dataframe. 
# However, when you specify a column to the groupby e.g. groupby(["X1", "X2"])["X3", "X4"], you basically drop all columns and leave only "X3" and "X4". But that is not the right way to use groupby
# When you specify a method to the groupby e.g. groupby(["X1", "X2"])["X3", "X4"].value_counts(), you basically make "X1" and "X2" the new indexes, and the method is applied to "X3" and "X4". 
# End result is X1 and X2 are indexes and the variables are the values derived from applying the method to X3 and X4 (X3 and X4 will not be the variables)

# For this case, we should use count because it simply adds up all the times a specific user appears. 
# For value_counts, it will aggregate only when both user and rating columns show the same value. Hence, it is not what we want since we are only looking for users who reviewed more than 200 books regardless of their ratings
# You can use the below to confirm the value_counts() output

user_RatingCount = df_total.groupby('user')['rating'].count().reset_index().rename(columns = {'rating':'userTotalRatingCount'})
# user_RatingCount = df_total.groupby(['user'])['rating'].value_counts().rename(index = 'count').reset_index()

book_RatingCount = df_total.groupby('isbn')['rating'].count().reset_index().rename(columns = {'rating':'bookTotalRatingCount'})
user_RatingCount.head()
book_RatingCount.head()

,isbn,bookTotalRatingCount
0,0000913154,1
1,0001010565,2
2,0001046438,1
3,0001046713,1
4,000104687X,1


In [ ]:
# Next we merge back the columns 

df_total = df_total.merge(user_RatingCount, how = 'left', left_on='user', right_on='user') 
df_total = df_total.merge(book_RatingCount, how = 'left',  left_on='isbn', right_on='isbn')
# Left means that we use df_total as the base dataframe and the Rating Count is mapped to df_total 'user' column. 
# The default 'inner' means that we only use the overlapping values between df_total and RatingCount. 

df_total.head()

,isbn,title,author,user,rating,userTotalRatingCount,bookTotalRatingCount
0,0195153448,Classical Mythology,Mark P. O. Morford,2,0.0,1,1
1,0002005018,Clara Callan,Richard Bruce Wright,8,5.0,17,14
2,0002005018,Clara Callan,Richard Bruce Wright,11400,0.0,62,14
3,0002005018,Clara Callan,Richard Bruce Wright,11676,8.0,11144,14
4,0002005018,Clara Callan,Richard Bruce Wright,41385,0.0,10,14


In [ ]:
# Now we can apply a concurrent filter to get only users with >200 reviews and books with >100 reviews

df_total = df_total.loc[(df_total['userTotalRatingCount'] > 200) & (df_total['bookTotalRatingCount'] > 100)]

In [ ]:
# Now we will design the matrix required to train our model
# We will just use the user-title system where our variables are specific users and their own ratings to the books they have picked  
# KNN model uses a csr matrix for input, so we need to prepare our csr matrix using the following steps
# CSR stands for compressed sparse row (CSR) or compressed row storage (CRS). A matrix M by three (one-dimensional) arrays, that respectively contain nonzero values, the extents of rows, and column indices.
# Steps include 1) Drop duplicates 2) Pivot the array from long to wide 3) Convert wide matrix into csr matrix

df_total_new = df_total.drop_duplicates(['user', 'title']) # Specifying 2 variables here means both variables must have the same value in order to trigger the drop in duplicates
# First we need to drop the duplicates as duplicates prevent the pivoting of the matrix.  Each user-title must have its own unique value for the conversion to work
# The default of drop duplicates is to assume the first as the unique value and subsequent ones as duplicates. You can change that to last or False depending on your dataset

df_total_pivot = df_total_new.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0) 
# Pivot it from long dataframe to wide dataframe, with all the columns being each user and the rows being each book
# Note that there will be a lot of null values simply because one user does not read every book, so you have to fill those null values. In this case, a 0 suffices as it just means no rating

df_total_matrix = csr_matrix(df_total_pivot.values)
# CSR matrix is created. You can see on the left is the row and col values for each individual cell, and on the right is the data found within that cell
print(df_total_matrix)

  (0, 0)	9.0
  (0, 34)	9.0
  (0, 38)	8.0
  (0, 55)	7.0
  (0, 124)	9.0
  (0, 142)	10.0
  (0, 154)	9.0
  (0, 171)	8.0
  (0, 202)	10.0
  (0, 203)	10.0
  (0, 225)	10.0
  (0, 252)	10.0
  (0, 353)	10.0
  (0, 452)	9.0
  (0, 494)	10.0
  (0, 538)	9.0
  (0, 627)	9.0
  (0, 692)	8.0
  (0, 699)	7.0
  (0, 797)	10.0
  (1, 9)	9.0
  (1, 23)	10.0
  (1, 38)	9.0
  (1, 43)	10.0
  (1, 74)	8.0
  :	:
  (662, 212)	8.0
  (662, 247)	6.0
  (662, 298)	9.0
  (662, 391)	9.0
  (662, 425)	10.0
  (662, 582)	8.0
  (662, 595)	7.0
  (662, 678)	7.0
  (662, 690)	9.0
  (663, 14)	8.0
  (663, 23)	7.0
  (663, 103)	5.0
  (663, 136)	6.0
  (663, 213)	7.0
  (663, 264)	8.0
  (663, 275)	9.0
  (663, 293)	10.0
  (663, 333)	10.0
  (663, 348)	9.0
  (663, 444)	10.0
  (663, 450)	8.0
  (663, 527)	8.0
  (663, 645)	8.0
  (663, 757)	9.0
  (663, 801)	8.0


In [ ]:
model = NearestNeighbors(algorithm='brute', metric='cosine') # You can see the distancemetric website on SKlearn for the full list. In this case, we shall just go with cosine, which will give distances between -1 and 1
model.fit(df_total_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  
  # First get the index of the book from the pivoted table
  # Another method is
  # X = books_with_ratings_pivot[books_with_ratings_pivot.index == book]
  # X = X.to_numpy().reshape(1,-1)
  # distances, indices = model_knn.kneighbors(X,n_neighbors=5)
  for query_index in range(len(df_total_pivot.index)):
    if df_total_pivot.index[query_index] == book:
      break
  
  # Next run the prediction model
  # For the model input, you just need to input the row values for that queried book, which is indicated by the  
  distances, indices = model.kneighbors(df_total_pivot.iloc[query_index, :].values.reshape(1,-1), n_neighbors = 6)
   
  recommended_books = [book, [ [],[],[],[],[] ] ]

  for i in range(1, len(distances.flatten())):
    recommended_books[1][i-1].append(df_total_pivot.index[indices.flatten()[len(distances.flatten()) - i]]) # The indices return in the form of [[X,Y,Z]], so you have to flatten it first and then use position i to single out the index to be used 
    recommended_books[1][i-1].append(distances.flatten()[len(distances.flatten()) - i]) # Similarly, distances return in the form of [[X,Y,Z]], so have to flatten it out first

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Icy Sparks', 0.7932764], ['Tis: A Memoir', 0.78800046], ['The Weight of Water', 0.76140916], ['I Know This Much Is True', 0.7581284], ['The Lovely Bones: A Novel', 0.72582525]]]
You haven't passed yet. Keep trying!
